In [1]:
# Google Earth Engine packages
import ee
import geemap

# other packages
import pandas as pd
import numpy as np

In [2]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()

In [3]:
filename = 'output/srtm_gee.tif'
print(filename)

output/srtm_gee.tif


In [4]:
# call map that centers aroung the region
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
"""
image = ee.Image('CGIAR/SRTM90_V4')
srtm_vis = { 'bands': 'elevation'}
#elevation = dataset.select('elevation')
#slope = ee.Terrain.slope(elevation)

Map.addLayer(image, srtm_vis, 'SRTM')
"""

In [17]:
image = ee.Image('USGS/SRTMGL1_003')
srtm_vis = { 'bands': 'elevation'}

Map.addLayer(image, srtm_vis, 'SRTM_NASA')

In [ ]:
#feature = Map.draw_last_feature
#roi = feature.geometry()

In [18]:
region = ee.Geometry.Polygon(
        [[[77.9818, 42.47152],
          [77.9818, 41.99486],
          [78.4801, 41.99486],
          [78.4801, 42.47152]]])
# add region to map
Map.addLayer(region, {}, 'Region')
Map.centerObject(region, zoom=9)

In [19]:
geemap.ee_export_image(image, filename=filename, scale=30, region=region, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to C:\Python\matilda_edu\output


In [20]:
import matplotlib.pyplot as plt
from pysheds.grid import Grid
import fiona

#Define a function to plot the digital elevation model
def plotFigure(data, label, cmap='Blues'):
    plt.figure(figsize=(12,10))
    plt.imshow(data, extent=grid.extent)
    plt.colorbar(label=label)
    plt.grid()

In [26]:
output_file = 'output/srtm_gee.shp'
y, x = 42.16268596, 78.26632477 # hydro Kashkator

In [27]:
DEM_file = filename

# Plot the DEM
grid = Grid.from_raster(DEM_file)
dem = grid.read_raster(DEM_file)
grid.view(dem)

# Fill depressions in DEM
flooded_dem = grid.fill_depressions(dem)
# Resolve flats in DEM
inflated_dem = grid.resolve_flats(flooded_dem)

# Specify directional mapping
#N    NE    E    SE    S    SW    W    NW
dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
# Compute flow directions
fdir = grid.flowdir(inflated_dem, dirmap=dirmap)

catch = grid.catchment(x=x, y=y, fdir=fdir, dirmap=dirmap, xytype='coordinate')

# Clip the DEM to the catchment
grid.clip_to(catch)
clipped_catch = grid.view(catch)


demView = grid.view(dem, nodata=np.nan)
plotFigure(demView,'Elevation')
plt.show()

In [28]:
## Create shapefile and save it
shapes = grid.polygonize()

schema = {
    'geometry': 'Polygon',
    'properties': {'LABEL': 'float:16'}
}

with fiona.open(output_file, 'w',
                driver='ESRI Shapefile',
                crs=grid.crs.srs,
                schema=schema) as c:
    i = 0
    for shape, value in shapes:
        rec = {}
        rec['geometry'] = shape
        rec['properties'] = {'LABEL' : str(value)}
        rec['id'] = str(i)
        c.write(rec)
        i += 1
c.close()